In [70]:
import pandas as pd
import numpy as np

In [71]:
from etl import extract as x

In [72]:
input_path = 'data/in/lotteria.html'
df = x.get_df_from_html(new_var)

In [73]:
df[df.Numero.isna()]

,Categoria,Serie,Numero,Luogo,Prov,Premio
52,3,L,NaN,RIMINI,RN,50000
53,3,S,NaN,NAPOLI,NA,50000
54,3,N,NaN,BENTIVOGLIO,BO,50000
55,3,T,NaN,GIUSSANO,MB,50000
56,3,M,NaN,PALERMO,PA,50000
...,...,...,...,...,...,...
275,4,N,NaN,TORINO,TO,20000
276,4,M,NaN,DISTRIBUTORE LOCALE ROMA,RM,20000
277,4,T,NaN,ROMA,RM,20000
278,4,Q,NaN,BAGHERIA,PA,20000


In [74]:
tables = pd.read_html(input_path)
tables

[       0       1         2      3          4
 0  Serie  Numero  Località  Prov.     Premio
 1      T  173756  Somaglia     LO  5.000.000
 2      T  378442    Pesaro     PU  2.500.000
 3      G  330068   Palermo     PA  2.000.000
 4      G  173817    Torino     TO  1.500.000
 5      S  185025      Dolo     VE  1.000.000,
      0                  1                  2                                               3                  4   5
 0  NaN  SECONDA CATEGORIA  SECONDA CATEGORIA                               SECONDA CATEGORIA  SECONDA CATEGORIA NaN
 1  NaN              Serie          Biglietto                                       Venduto a             Premio NaN
 2  NaN                  S             030585  DISTRIBUTORE LOCALE MONTENERO DI BISACCIA (CB)            100.000 NaN
 3  NaN                  C             439458                                   AVELLINO (AV)            100.000 NaN
 4  NaN                  M             217442                                   MASSAFRA (TA

In [75]:
tbl1 = tables[0]
col_names = tbl1.iloc[0]                        # no Categoria
cat1 = x.process_cat1(tbl1, col_names)
cat1

,Categoria,Serie,Numero,Località,Prov.,Premio
1,1,T,173756,Somaglia,LO,5000000
2,1,T,378442,Pesaro,PU,2500000
3,1,G,330068,Palermo,PA,2000000
4,1,G,173817,Torino,TO,1500000
5,1,S,185025,Dolo,VE,1000000


# OK

In [76]:
x.drop_all_nan_cols(tables)


In [77]:
rows = x.parse_tables(tables)

# create_newdf

In [78]:
newdf = pd.DataFrame(                                       # create dataframe from rows
                rows, 
                columns=[
                    'Categoria', 
                    'Serie', 
                    'Numero', 
                    'Località', 
                    'Premio'
                ]
)


## newdf.Numero ok

In [79]:
newdf.Numero.isna().any()

np.False_

In [80]:
newdf.insert(                                               # insert prov column
            4, 
            'Prov.', 
            newdf['Località'].str.extract(r'\((.*)\)')      # extract (PROV) from location (regex)
)


In [81]:
newdf['Località'] = newdf['Località'].apply(                # remove (PROV) from location
                    lambda x: x.split('(')[0]
)


In [82]:
newdf['Premio'] = newdf['Premio'].astype('str')             # convert premio to str


In [83]:
newdf['Premio'] = newdf.Premio.apply(                       # fix zeroes
                    lambda x: x + '00' 
                    if x.endswith('.0') 
                    else x
)


In [84]:
newdf.Premio = (                                            # convert premio to int
            newdf.Premio.str.replace('.', '')
            .astype('int')
)


In [85]:
cat1.head()

,Categoria,Serie,Numero,Località,Prov.,Premio
1,1,T,173756,Somaglia,LO,5000000
2,1,T,378442,Pesaro,PU,2500000
3,1,G,330068,Palermo,PA,2000000
4,1,G,173817,Torino,TO,1500000
5,1,S,185025,Dolo,VE,1000000


In [86]:
newdf.head()

,Categoria,Serie,Numero,Località,Prov.,Premio
0,2,S,030585,DISTRIBUTORE LOCALE MONTENERO DI BISACCIA,CB,100000
1,2,C,439458,AVELLINO,AV,100000
2,2,M,217442,MASSAFRA,TA,100000
3,2,D,138065,FOLIGNO,PG,100000
4,2,M,442388,BELLUNO,BL,100000


# OK

In [87]:
winners = pd.concat([cat1, newdf]).reset_index(drop=True)

In [88]:
winners = winners.reset_index(drop=True)

In [89]:
winners = winners.rename(columns={
    'Prov.':'Prov',
    'Località': 'Luogo'
})

# PROBLEM

In [93]:
winners.Numero.isna().value_counts()

Numero
False    280
Name: count, dtype: int64

# FIXED

In [115]:
for col in ['Serie', 'Numero', 'Luogo', 'Prov']:
    winners[col] = winners[col].astype('str').str.upper().str.rstrip()